In [ ]:
import pickle

In [ ]:
import torch

with open("../feed_dict.json","rb") as file:
    feed_dict = pickle.load(file)

In [ ]:
feed_dict_q = feed_dict
feed_dict_k = feed_dict

In [ ]:
import torch

feed_dict_q["view"] = torch.rand((feed_dict["image"].size(0),1,7))
feed_dict_k["view"] = torch.rand((feed_dict["image"].size(0),1,7))

feed_dict["images"] = feed_dict["image"]
feed_dict["objects_boxes"] = feed_dict["objects"]
feed_dict["objects"] = feed_dict["objects_length"]

In [ ]:
import sys
sys.path.append("../")

In [ ]:
# default_exp model.encoder

In [ ]:
#export

from core.scene_graph.scene_graph import SceneGraph
from torchvision.models import resnet34
import torch.nn as nn
import torch

In [ ]:
#export

class Encoder(nn.Module):
    def __init__(self, dim = 256):
        super().__init__()
        
        """
        Input:
            dim : final number of dimensions of the node and spatial embeddings
        
        Returns:
            Intialises a model which has node embeddimgs and spatial embeddings
        """
        
        
        self.dim = dim
        self.resnet = resnet34(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(self.resnet.children())[:-3])
        
        self.scene_graph = SceneGraph(feature_dim=self.dim, 
                                 output_dims=[self.dim,self.dim],
                                 downsample_rate=16)
        
        
    def forward(self,feed_dict):
        """
        Input:
            feed_dict: a dictionary containing list tensors containing images and bounding box data. 
            Each element of the feed_dict corresponds to one elment of the batch. 
            Inside each batch are contained ["image": Image tensor, 
                                             "boxes":Bounding box tensor,
                                             bounding box
                                            ]
        """
        num_batch = feed_dict["images"].shape[0]
        num_total_nodes = feed_dict["objects"].sum().item()
        
        image_features = self.feature_extractor(feed_dict["images"])
        outputs = self.scene_graph(image_features, feed_dict["objects_boxes"], feed_dict["objects"])
        
        node_features = outputs[0][0]
        for num in range(1,num_batch):
            node_features = torch.cat([node_features, outputs[num][0]], dim =0)
        
        # To be implemented
        spatial_features = None
        
        return outputs, node_features, spatial_features

In [ ]:
encoder = Encoder()

In [ ]:
output, node_features, spatial_features = encoder(feed_dict)

In [ ]:
node_features.shape

torch.Size([13, 256])

In [ ]:
feed_dict["images"].shape

torch.Size([2, 3, 256, 384])

In [ ]:
torch.zeros(1, dtype=torch.long)

tensor([0])